#### Things to know
- - -

##### List of the classification models
* Decision Tree (Gini, Entropy)
* Soft Vector Machine
* K Nearnest Neighbors
* Random Forest
* Gradient Boosting
* Logistic Regression
* Simple Neural Network
* Naive Bayesian

- - -

##### 1. import required libs

In [ ]:
import sklearn.tree
import sklearn.svm
import sklearn.ensemble
import sklearn.neighbors
import sklearn.linear_model
import sklearn.gaussian_process
import sklearn.preprocessing
import sklearn.neural_network
import sklearn.model_selection

In [ ]:
### LIB TO OPEN EXCEL FILE
import pandas as pd
from matplotlib import pyplot as plt
from pathlib import Path
import joblib

##### 2. load dataset

In [ ]:
import numpy as np
source_quantile = "0.2"
data_file = f'all_pids_q={source_quantile}'

ROOT_DIR = '../../'

output_dir = f"{ROOT_DIR}Output_Results/"
data_dir = f'{ROOT_DIR}Input_Data/Og_Data/labeled_normal_data/'

data = np.load(data_dir + data_file + '.npy', allow_pickle=True).item()

In [ ]:
print(len(data))
data.keys()

##### 3. set target patient's id

In [ ]:
target_id = 'SB-003'
df_origin = data[target_id]
df_origin.keys()

##### 4. remove columns which is not needed for classification

In [ ]:
df = df_origin.copy()
drop_attributes = ['id', 'sex', 'age','am_pef_org','BMI']
df = df.drop(drop_attributes, axis=1)
df.sort_values(by='date', ascending=True, inplace=True)
pefr = df['y_am_pef']
df = df.drop(['date'], axis=1) #, 'y_am_pef'
display(df.head(5), df.tail(5))

In [ ]:
##### KEY TROUBLESHOOTING NOTE
##### `np.load` returns a `numpy ndarray`
#data_ndarr = np.load(ROOT_DIR / source_file_path, allow_pickle=True)
data_ndarr = np.load(data_dir + data_file + '.npy', allow_pickle=True)
print(type(data_ndarr))

##### That array has a method `item()`,
##### which returns a *dictionary* of key-values: `p_id`:`DataFrame`
datadict = data_ndarr.item()
print(type(datadict))

In [ ]:
p_ids = datadict.keys()

print("\t\t\t\tpatients: n =", len(p_ids), end='\n\n')

for p_id in p_ids:
    df = datadict[p_id]
    # drop column from input data file
    #df.drop(['id','date','am_pef_org','BMI','sex'], axis=1, inplace=True) #ALL DROPS SHOULD OCCUR HERE
    df.drop(['id','date','am_pef_org','BMI','sex', 'age'], axis=1, inplace=True) #,'id','date','am_pef_org','BMI',  'sex', 'age'
    df = np.asarray(df).astype(np.float32)
    print(p_id, "shape:", df.shape, end="\n")

datadict[p_id].head(4)

##### 5. import sada's function for classification

In [ ]:
import importlib.util
spec = importlib.util.spec_from_file_location('sada.classification', 'Classificationv3.py')
sada = importlib.util.module_from_spec(spec)
spec.loader.exec_module(sada)

##### 6. set classification models to use

In [ ]:
df = df_origin.copy()
drop_attributes = ['id', 'sex', 'age','am_pef_org','BMI']
df = df.drop(drop_attributes, axis=1)
df.sort_values(by='date', ascending=True, inplace=True)
pefr = df['y_am_pef']
df = df.drop(['date'], axis=1) #, 'y_am_pef'
display(df.head(5), df.tail(5))

In [ ]:
predictColumn = "class"
y = df[predictColumn]
#X = df.drop(predictColumn, axis=1)
class_names = y.unique()
class_names[::-1].sort()
n_classes = len(class_names)
if n_classes > 2 :
    logit_reg_model = sklearn.linear_model.LogisticRegression(solver='saga', multi_class='multinomial')
else :
    logit_reg_model = sklearn.linear_model.LogisticRegression(solver='liblinear', multi_class='auto')

cls = [# Comment/uncomment out any classfiers according to your need
        # DO NOT CHANGE
        #"DecisionTree Gini" , sklearn.tree.DecisionTreeClassifier(max_depth=4, criterion="gini"),
        #"DecisionTree Entr" , sklearn.tree.DecisionTreeClassifier(max_depth=4, criterion="entropy"),
        "DecisionTree" , sklearn.tree.DecisionTreeClassifier(max_depth=4),
        "SVM"               , sklearn.svm.SVC(kernel="linear"),
#        "SVM1"              , SVC(kernel="rbf", C=10**3),
        "Random Forest"     , sklearn.ensemble.RandomForestClassifier(n_estimators=10),
        "K-NN"              , sklearn.neighbors.KNeighborsClassifier(),
        "Gradient Boosting" , sklearn.ensemble.GradientBoostingClassifier(),
        "Logit Regression"  , logit_reg_model,
        "Neural NW"         , sklearn.neural_network.MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(32, 16), max_iter=10000, random_state=1),
        "Naive Bayes"       , sklearn.gaussian_process.GaussianProcessClassifier()
];

from sklearn.preprocessing import StandardScaler
kf = sklearn.model_selection.KFold(n_splits=5,shuffle=True,random_state=26)
y_pred = y.copy()
l = sklearn.preprocessing.LabelEncoder()
encoded_y = l.fit_transform(y);
scaler = StandardScaler()
scaled_X = scaler.fit_transform(X)
print(type(scaled_X))
new_df = pd.DataFrame(scaled_X)
new_df = new_df.assign(label = list(encoded_y))
print(new_df)
balanced_new_df = wade.BalanceClassesWithDuplicates(new_df, 'label')
asdf_y = 

In [ ]:
output_dir = f"{output_dir}Classification_NN_q={str(source_quantile)}_Og_Data_{target_id}_Evaluation"

print(output_dir)

import os
if not os.path.isdir(Path(output_dir)):
    os.makedirs(Path(output_dir))

##### 7. classify the data and save the results

In [ ]:
target_grp = ["SB-028", "SB-011", "SB-112", "SB-001", "SB-012", "SB-073", "SB-078", "SB-080", "SB-083", "SB-043", "SB-037", "SB-022", "SB-025", "SB-060", "SB-023", "SB-071", "SB-033", "SB-079", "SB-056", "SB-059", "SB-003", "SB-089", "SB-082", "SB-008", "SB-070"]
EVALUATION_METRICS = ['Weighted Accuracy', 'Sensitivity/Recall', 'Precision_avg', 'F1_avg', 'Accuracy', 'Specificity','Precision', 'Precision_class1', 'F1', 'F1_class1', 'Dice', 'Jaccard']

In [ ]:
def classify(category_group, cls, target_id, target_grp, output_dir, df_evaluation_results):
    #runtime()

    if len(cls)!=2:
        print('process one model each time')
        return;

    print('~~~~~~~~~~~~~~~~~', target_id,'~~~~~~~~~~~~~~~~~~~~~~')
    patient_dir = Path(f"{output_dir}{category_group}diagrams/") #RESULT_DIR + "diagrams" + target  #

    print(patient_dir)

    # create directory for this patient
    if not (patient_dir.exists() and patient_dir.is_dir()):
        patient_dir.mkdir()

    save_path = f'{patient_dir}/balanced_for_train_'
    print(save_path)

    CLASS_VAR = 'class'
    bestDT = sada.Classify(datadict, target_id, True, class_var=CLASS_VAR, drawConfusionMatrix=True, classifiers=cls, scale=True, save_path=save_path, after_split=True)

    confusion_matrixes_file = (
        f"{save_path}confusion_mat_{category_group}_{target_id}"
    )
    sada.save_confusion_matrix(bestDT[1], confusion_matrixes_file)
    roc_curve_file = f"{save_path}roc_curve_{category_group}_{target_id}"
    sada.save_Draw_Roc(bestDT[2], roc_curve_file)
    if len(bestDT[3])!=0:
        featureImportance_path = (
            f"{save_path}important_mat_{category_group}_{target_id}"
        )
        sada.save_Draw_FeatureImportanceMatrix(datadict[target_id],bestDT[3],cls[0], featureImportance_path, True)

    model_path = f"{save_path}bestModel_{category_group}_{target_id}.joblib"
    joblib.dump(bestDT[4], model_path)
    loaded_model = joblib.load(model_path)
    print(loaded_model)

    df_evaluation_results[target_id] = bestDT[0]
    df_evaluation_results.to_csv(
        f"{output_dir}/balanced_for_train_metrics_{category_group}.csv"
    )

### 1. Tuning DecisionTree

In [ ]:
df_evaluation_results = pd.DataFrame(index=EVALUATION_METRICS)
category_group = '1DecisionTree'
cls = [# Comment/uncomment out any classfiers according to your need
        # DO NOT CHANGE
        "DecisionTree" , sklearn.tree.DecisionTreeClassifier(max_depth=4),
        #"SVM"               , sklearn.svm.SVC(kernel="linear"),
        #"Random Forest"     , sklearn.ensemble.RandomForestClassifier(n_estimators=10),
        #"K-NN"              , sklearn.neighbors.KNeighborsClassifier(),
        #"Gradient Boosting" , sklearn.ensemble.GradientBoostingClassifier(),
        #"Logit Regression"  , logit_reg_model,
        #"Neural NW"         , sklearn.neural_network.MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(32, 16), max_iter=10000, random_state=1),
        #"Naive Bayes"       , sklearn.gaussian_process.GaussianProcessClassifier()
];

#target_id = 'SB-003'

for target_id in p_ids:
    if target_id not in target_grp:
        continue
    #runtime()
    classify(category_group, cls, target_id, target_grp, output_dir, df_evaluation_results)

### 2. SVM

In [ ]:
df_evaluation_results = pd.DataFrame(index=EVALUATION_METRICS)
category_group = '2SVM'
cls = [# Comment/uncomment out any classfiers according to your need 
       # DO NOT CHANGE
        #"DecisionTree" , sklearn.tree.DecisionTreeClassifier(max_depth=4),
        "SVM"               , sklearn.svm.SVC(kernel="linear"), 
        #"Random Forest"     , sklearn.ensemble.RandomForestClassifier(n_estimators=10),
        #"K-NN"              , sklearn.neighbors.KNeighborsClassifier(),
        #"Gradient Boosting" , sklearn.ensemble.GradientBoostingClassifier(),
        #"Logit Regression"  , logit_reg_model,
        #"Neural NW"         , sklearn.neural_network.MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(32, 16), max_iter=10000, random_state=1),
        #"Naive Bayes"       , sklearn.gaussian_process.GaussianProcessClassifier()
];
target_id = 'SB-003'

classify(category_group, cls, target_id, target_grp, output_dir, df_evaluation_results)



### 3. Random Forest

In [ ]:
df_evaluation_results = pd.DataFrame(index=EVALUATION_METRICS)
category_group = '3RandomForest'
cls = [# Comment/uncomment out any classfiers according to your need 
       # DO NOT CHANGE
        #"DecisionTree" , sklearn.tree.DecisionTreeClassifier(max_depth=4),
        #"SVM"               , sklearn.svm.SVC(kernel="linear"), 
        "Random Forest"     , sklearn.ensemble.RandomForestClassifier(n_estimators=10),
        #"K-NN"              , sklearn.neighbors.KNeighborsClassifier(),
        #"Gradient Boosting" , sklearn.ensemble.GradientBoostingClassifier(),
        #"Logit Regression"  , logit_reg_model,
        #"Neural NW"         , sklearn.neural_network.MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(32, 16), max_iter=10000, random_state=1),
        #"Naive Bayes"       , sklearn.gaussian_process.GaussianProcessClassifier()
];
target_id = 'SB-003'

classify(category_group, cls, target_id, target_grp, output_dir, df_evaluation_results)
    


### 4. K-NN

In [ ]:
df_evaluation_results = pd.DataFrame(index=EVALUATION_METRICS)
category_group = '4K-NN'
cls = [# Comment/uncomment out any classfiers according to your need 
       # DO NOT CHANGE
        #"DecisionTree" , sklearn.tree.DecisionTreeClassifier(max_depth=4),
        #"SVM"               , sklearn.svm.SVC(kernel="linear"), 
        #"Random Forest"     , sklearn.ensemble.RandomForestClassifier(n_estimators=10),
        "K-NN"              , sklearn.neighbors.KNeighborsClassifier(),
        #"Gradient Boosting" , sklearn.ensemble.GradientBoostingClassifier(),
        #"Logit Regression"  , logit_reg_model,
        #"Neural NW"         , sklearn.neural_network.MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(32, 16), max_iter=10000, random_state=1),
        #"Naive Bayes"       , sklearn.gaussian_process.GaussianProcessClassifier()
];
target_id = 'SB-003'

classify(category_group, cls, target_id, target_grp, output_dir, df_evaluation_results)

### 5. Gradient Boosting

In [ ]:
df_evaluation_results = pd.DataFrame(index=EVALUATION_METRICS)
category_group = '5GradientBoosting'
cls = [# Comment/uncomment out any classfiers according to your need 
       # DO NOT CHANGE
        #"DecisionTree" , sklearn.tree.DecisionTreeClassifier(max_depth=4),
        #"SVM"               , sklearn.svm.SVC(kernel="linear"), 
        #"Random Forest"     , sklearn.ensemble.RandomForestClassifier(n_estimators=10),
        #"K-NN"              , sklearn.neighbors.KNeighborsClassifier(),
        "Gradient Boosting" , sklearn.ensemble.GradientBoostingClassifier(),
        #"Logit Regression"  , logit_reg_model,
        #"Neural NW"         , sklearn.neural_network.MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(32, 16), max_iter=10000, random_state=1),
        #"Naive Bayes"       , sklearn.gaussian_process.GaussianProcessClassifier()
];
target_id = 'SB-003'

classify(category_group, cls, target_id, target_grp, output_dir, df_evaluation_results)


### 6. Logit Regression

In [ ]:
df_evaluation_results = pd.DataFrame(index=EVALUATION_METRICS)
category_group = '6LogitRegression'
cls = [# Comment/uncomment out any classfiers according to your need 
       # DO NOT CHANGE
        #"DecisionTree" , sklearn.tree.DecisionTreeClassifier(max_depth=4),
        #"SVM"               , sklearn.svm.SVC(kernel="linear"), 
        #"Random Forest"     , sklearn.ensemble.RandomForestClassifier(n_estimators=10),
        #"K-NN"              , sklearn.neighbors.KNeighborsClassifier(),
        #"Gradient Boosting" , sklearn.ensemble.GradientBoostingClassifier(),
        "Logit Regression"  , logit_reg_model,
        #"Neural NW"         , sklearn.neural_network.MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(32, 16), max_iter=10000, random_state=1),
        #"Naive Bayes"       , sklearn.gaussian_process.GaussianProcessClassifier()
];
target_id = 'SB-003'

classify(category_group, cls, target_id, target_grp, output_dir, df_evaluation_results)


### 7. Neural Network

In [ ]:
df_evaluation_results = pd.DataFrame(index=EVALUATION_METRICS)
category_group = '7NeuralNW'
cls = [# Comment/uncomment out any classfiers according to your need 
       # DO NOT CHANGE
        #"DecisionTree" , sklearn.tree.DecisionTreeClassifier(max_depth=4),
        #"SVM"               , sklearn.svm.SVC(kernel="linear"), 
        #"Random Forest"     , sklearn.ensemble.RandomForestClassifier(n_estimators=10),
        #"K-NN"              , sklearn.neighbors.KNeighborsClassifier(),
        #"Gradient Boosting" , sklearn.ensemble.GradientBoostingClassifier(),
        #"Logit Regression"  , logit_reg_model,
        "Neural NW"         , sklearn.neural_network.MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(32, 16), max_iter=10000, random_state=1),
        #"Naive Bayes"       , sklearn.gaussian_process.GaussianProcessClassifier()
];
target_id = 'SB-003'

classify(category_group, cls, target_id, target_grp, output_dir, df_evaluation_results)


### 8. Naive Bays

In [ ]:
df_evaluation_results = pd.DataFrame(index=EVALUATION_METRICS)
category_group = '8NaiveBayes'
cls = [# Comment/uncomment out any classfiers according to your need 
       # DO NOT CHANGE
        #"DecisionTree" , sklearn.tree.DecisionTreeClassifier(max_depth=4),
        #"SVM"               , sklearn.svm.SVC(kernel="linear"), 
        #"Random Forest"     , sklearn.ensemble.RandomForestClassifier(n_estimators=10),
        #"K-NN"              , sklearn.neighbors.KNeighborsClassifier(),
        #"Gradient Boosting" , sklearn.ensemble.GradientBoostingClassifier(),
        #"Logit Regression"  , logit_reg_model,
        #"Neural NW"         , sklearn.neural_network.MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(32, 16), max_iter=10000, random_state=1),
        "Naive Bayes"       , sklearn.gaussian_process.GaussianProcessClassifier()
];
target_id = 'SB-003'

classify(category_group, cls, target_id, target_grp, output_dir, df_evaluation_results)